In [1]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
import torch
from tqdm.autonotebook import tqdm
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset

/home/sanjaymoto75/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
def frame_extract(path):
    vidObj = cv2.VideoCapture(path) 
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            yield image

In [3]:
import pandas as pd
train_label = pd.read_csv('./TrainLabels.csv')
valid_label = pd.read_csv('./ValidationLabels.csv')
test_label = pd.read_csv('./TestLabels.csv')

In [4]:
label0 = []
clipid0 = []
label1 = []
clipid1 = []

for i in range(len(train_label)):
    if(train_label['Engagement'][i]<2):
        clipid0.append(train_label['ClipID'][i])
    if(train_label['Engagement'][i]>=2):
        clipid1.append(train_label['ClipID'][i])

In [5]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
from tqdm.autonotebook import tqdm
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys
im_size = 112
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor()])

In [6]:
class video_dataset(Dataset):
    def __init__(self,frame_dir,clipid,transform = None):
        self.folder = os.listdir(frame_dir)
        self.id = clipid
        self.frame_dir = frame_dir
        self.transform = transform
    def __len__(self):
        return len(self.id)
    def __getitem__(self,idx):
        id_1 = self.id[idx][:6]
        path1 = os.path.join(self.frame_dir,id_1)
        id_2 = self.id[idx][:-4]
        path2 = os.path.join(path1,id_2)
        seq_image = list()
        i = 0
        path3 = os.path.join(path2,str(i)+'.jpg')
        image = cv2.imread(path3)
        face_locations = face_recognition.face_locations(image)
        flag = 0
        if(len(face_locations) == 0):
            flag = 1
        else:
            top,right,bottom,left = face_locations[0]
            top = int(top*0.7)
            left = int(left*0.6)
            right = int(right*1.3)
        all_images = []
        count = 0
        while i<300:
            count = count+1
            path3 = os.path.join(path2,str(i)+'.jpg')
            image = cv2.imread(path3)
            if(flag==1):
                face_locations = face_recognition.face_locations(image)
                if(len(face_locations)>0):
                    top,right,bottom,left = face_locations[0]
                    flag = 0
                    top = int(top*0.7)
                    left = int(left*0.6)
                    right = int(right*1.3)
                    image = image[top:,left:right,:]
                    if(self.transform):
                        image = self.transform(image)
                    seq_image.append(image)
                    if(len(seq_image)==16):
                        all_images.append(torch.stack(seq_image))
                        seq_image = []
                    i = i+2
                    continue
                if(self.transform):
                    image = self.transform(image)
                seq_image.append(image)
                if(len(seq_image)==16):
                    all_images.append(torch.stack(seq_image))
                    seq_image = []
                i = i+2
                continue
            image = image[top:,left:right,:]
            if(self.transform):
                image = self.transform(image)
            seq_image.append(image)
            if(len(seq_image)==16):
                all_images.append(torch.stack(seq_image))
                seq_image = []
            i = i+2
        all_images = torch.stack(all_images)
        all_images = all_images.permute((0,2,1,3,4))
        if(len(all_images)<9):
            print(id_1,idx)
        return all_images

In [7]:
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

In [8]:
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
train_data_0 = video_dataset('./frames/train',clipid0,transform = train_transforms)
train_data_1 = video_dataset('./frames/train',clipid1,transform = train_transforms)
#val_data = video_dataset('./frames/val',valid_label,sequence_length = 16,transform = test_transforms)
#train_loader = DataLoader(train_data,batch_size = 2,num_workers = 4 ,shuffle = True)
#valid_loader = DataLoader(val_data,batch_size = 2,num_workers = 4 ,shuffle = True)
#dataloaders = {'train':train_loader , 'val':valid_loader}

In [61]:
import cv2
import face_recognition
#image,_ = train_data[5000]
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.5,0.5,0.5] + [0.5,0.5,0.5]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()
#im_plot(image[:,10,:,:])

In [9]:
%cd ./representationflowcvpr19
import math

import rep_flow_layer as rf


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

import os
import sys

class SamePadding(nn.Module):

  def __init__(self, kernel_size, stride):
    super(SamePadding, self).__init__()
    self.kernel_size = kernel_size
    self.stride = stride

  def compute_pad(self, dim, s):
    if s % self.stride[dim] == 0:
      return max(self.kernel_size[dim] - self.stride[dim], 0)
    else:
      return max(self.kernel_size[dim] - (s % self.stride[dim]), 0)
    
  def forward(self, x):
    # compute 'same' padding
    (batch, channel, t, h, w) = x.size()
    #print t,h,w
    out_t = np.ceil(float(t) / float(self.stride[0]))
    out_h = np.ceil(float(h) / float(self.stride[1]))
    out_w = np.ceil(float(w) / float(self.stride[2]))
    #print out_t, out_h, out_w
    pad_t = self.compute_pad(0, t)
    pad_h = self.compute_pad(1, h)
    pad_w = self.compute_pad(2, w)
    #print pad_t, pad_h, pad_w
    
    pad_t_f = pad_t // 2
    pad_t_b = pad_t - pad_t_f
    pad_h_f = pad_h // 2
    pad_h_b = pad_h - pad_h_f
    pad_w_f = pad_w // 2
    pad_w_b = pad_w - pad_w_f
    
    pad = (pad_w_f, pad_w_b, pad_h_f, pad_h_b, pad_t_f, pad_t_b)
    #print x.size()
    #print pad
    x = F.pad(x, pad)
    return x


class Bottleneck3D(nn.Module):
  
  def __init__(self, inputs, filters, is_training, strides,
               use_projection=False, T=3, data_format='channels_last', non_local=False):
    """Bottleneck block variant for residual networks with BN after convolutions.
  Args:
    inputs: `Tensor` of size `[batch, channels, height, width]`.
    filters: `int` number of filters for the first two convolutions. Note that
        the third and final convolution will use 4 times as many filters.
    is_training: `bool` for whether the model is in training.
    strides: `int` block stride. If greater than 1, this block will ultimately
        downsample the input.
    use_projection: `bool` for whether this block should use a projection
        shortcut (versus the default identity shortcut). This is usually `True`
        for the first block of a block group, which may change the number of
        filters and the resolution.
    data_format: `str` either "channels_first" for `[batch, channels, height,
        width]` or "channels_last for `[batch, height, width, channels]`.
  Returns:
    The output `Tensor` of the block.
  """
    super(Bottleneck3D, self).__init__()
            
    df = 'NDHWC' if data_format == 'channels_last' else 'NCDHW'
    self.shortcut = None
    if use_projection:
      # Projection shortcut only in first block within a group. Bottleneck blocks
      # end with 4 times the number of filters.
      filters_out = 4 * filters
      self.shortcut = nn.Sequential(SamePadding((1,1,1),(1,strides,strides)),
                                    nn.Conv3d(
                                      inputs, filters_out, kernel_size=1, stride=(1,strides,strides), bias=False, padding=0),
                                    nn.BatchNorm3d(filters_out),
                                    nn.BatchNorm3d(filters_out)) # there are two, due to old models having it. To load weights, 2 batch norms are needed here...
    
    self.layers = nn.Sequential(SamePadding((T,1,1), (1,1,1)),
                                nn.Conv3d(inputs, filters, kernel_size=(T,1,1), stride=1, padding=(0,0,0), bias=False), #1
                                nn.BatchNorm3d(filters), #2
                                nn.ReLU(),
                                SamePadding((1,3,3),(1,strides,strides)),
                                nn.Conv3d(filters, filters, kernel_size=(1,3,3), stride=(1,strides,strides), bias=False, padding=0), #5
                                nn.BatchNorm3d(filters),#6
                                nn.ReLU(),
                                nn.Conv3d(filters, 4*filters, kernel_size=1, stride=1, bias=False, padding=0),#8
                                nn.BatchNorm3d(4*filters))#9
    

  def forward(self, x):
    #print('block', x.size())
    if self.shortcut:
      res = self.shortcut(x)
    else:
      res = x
    #print('b2',x.size())
    return F.relu(self.layers(x) + res)


  
class Block3D(nn.Module):
  def __init__(self, inputs, filters, block_fn, blocks, strides, is_training, name,
                   data_format='channels_last', non_local=0):
    """Creates one group of blocks for the ResNet model.
    
  Args:
    inputs: `Tensor` of size `[batch, channels, height, width]`.
    filters: `int` number of filters for the first convolution of the layer.
    block_fn: `function` for the block to use within the model
    blocks: `int` number of blocks contained in the layer.
    strides: `int` stride to use for the first convolution of the layer. If
        greater than 1, this layer will downsample the input.
    is_training: `bool` for whether the model is training.
    name: `str`name for the Tensor output of the block layer.
    data_format: `str` either "channels_first" for `[batch, channels, height,
        width]` or "channels_last for `[batch, height, width, channels]`.
  Returns:
    The output `Tensor` of the block layer.
    """
    super(Block3D, self).__init__()

    self.blocks = nn.ModuleList()
    # Only the first block per block_group uses projection shortcut and strides.
    self.blocks.append(Bottleneck3D(inputs, filters, is_training, strides,
                                    use_projection=True, data_format=data_format))
    inputs = filters * 4
    T = 3
    for i in range(1, blocks):
      self.blocks.append(Bottleneck3D(inputs, filters, is_training, 1, T=T,
                                      data_format=data_format, non_local=0))
      # only use 1 3D conv per 2 residual blocks (per Non-local NN paper)
      T = (3 if T==1 else 1)
    

  def forward(self, x):
    for block in self.blocks:
      x = block(x)
    return x

class ResNet3D(nn.Module):
  
  def __init__(self, block_fn, layers, num_classes,
               data_format='channels_last', non_local=[], rep_flow=[],
               dropout_keep_prob=0.5):
    """Generator for ResNet v1 models.
  Args:
    block_fn: `function` for the block to use within the model. Either
        `residual_block` or `bottleneck_block`.
    layers: list of 4 `int`s denoting the number of blocks to include in each
      of the 4 block groups. Each group consists of blocks that take inputs of
      the same resolution.
    num_classes: `int` number of possible classes for image classification.
    data_format: `str` either "channels_first" for `[batch, channels, height,
        width]` or "channels_last for `[batch, height, width, channels]`.
  Returns:
    Model `function` that takes in `inputs` and `is_training` and returns the
    output `Tensor` of the ResNet model.
    """
    super(ResNet3D, self).__init__()
    is_training = False # no effect in pytorch

    # TODO: RF Layer!
    #self.rep_flow = rep_flow_layer.rep_flow(inputs, rep_flow[0], is_training, bottleneck=1, data_format=data_format, use_last_conv=False)

    """Creation of the model graph."""
    self.stem = nn.Conv3d(
      3, 64, kernel_size=7, bias=False, stride=2)
    
    self.bn1 = nn.BatchNorm3d(64, eps=0.001, momentum=0.01)
    self.relu = nn.ReLU(inplace=True)
    self.pad = SamePadding((3,3,3),(2,2,2))
    self.maxpool = nn.MaxPool3d(kernel_size=3,
                                stride=2, padding=0)

    self.rep_flow = rf.FlowLayer(512)

    # res 2
    inputs = 64
    self.res2 = Block3D(
      inputs=inputs, filters=64, block_fn=block_fn, blocks=layers[0],
      strides=1, is_training=is_training, name='block_group1',
      data_format=data_format, non_local=non_local[0])
    
    # res 3
    inputs = 64*4
    self.res3 = Block3D(
      inputs=inputs, filters=128, block_fn=block_fn, blocks=layers[1],
      strides=2, is_training=is_training, name='block_group2',
      data_format=data_format, non_local=non_local[1])

    # res 4
    inputs = 128*4
    self.res4 = Block3D(
      inputs=inputs, filters=256, block_fn=block_fn, blocks=layers[2],
      strides=2, is_training=is_training, name='block_group3',
      data_format=data_format, non_local=non_local[2])

    # res 5
    inputs = 256*4
    self.res5 = Block3D(
        inputs=inputs, filters=512, block_fn=block_fn, blocks=layers[3],
        strides=2, is_training=is_training, name='block_group4',
        data_format=data_format, non_local=non_local[3])

    self.dropout = nn.Dropout(0.5)
    self.classify = nn.Conv3d(512*4, num_classes, kernel_size=1, stride=1)

  def forward(self, x):
    x = self.stem(x)
    x = self.bn1(x)
    x = F.relu(x)
    x = self.maxpool(self.pad(x))
    x = self.res2(x)
    x = self.res3(x)

    x = self.rep_flow(x)

    x = self.res4(x)
    x = self.res5(x)
    x = x.mean(3).mean(3).unsqueeze(3).unsqueeze(3) # spatial average
    x = self.dropout(x)
    x = self.classify(x)
    x = x.mean(2) # temporal average
    return x


def resnet_3d_v1(resnet_depth, num_classes, data_format='channels_last', is_3d=True, non_local=[0,0,0,0], rep_flow=[0,0,0,0,0]):
  """Returns the ResNet model for a given size and number of output classes."""
  model_params = {
      18: {'block': None, 'layers': [2, 2, 2, 2]},
      34: {'block': None, 'layers': [3, 4, 6, 3]},
      50: {'block': None, 'layers': [3, 4, 6, 3]},
      101: {'block': None, 'layers': [3, 4, 23, 3]},
      152: {'block': None, 'layers': [3, 8, 36, 3]},
      200: {'block': None, 'layers': [3, 24, 36, 3]}
  }

  if resnet_depth not in model_params:
    raise ValueError('Not a valid resnet_depth:', resnet_depth)

  params = model_params[resnet_depth]
  return ResNet3D(
    params['block'], params['layers'], num_classes, data_format, non_local, rep_flow)




net = resnet_3d_v1(50, 400)

/home/sanjaymoto75/attention/representationflowcvpr19


In [11]:
%cd /home/sanjaymoto75/attention/
from torchsummary import summary
net = resnet_3d_v1(50, 400).to('cuda')
net.load_state_dict(torch.load("./data.pt"))
net.classify = nn.Sequential(nn.Conv3d(512*4, 1, kernel_size=1, stride=1),nn.Sigmoid()).to('cuda')
summary(net,(3,16,112,112))
net.load_state_dict(torch.load('nn_crime.pt'))
model = nn.DataParallel(net)

/home/sanjaymoto75/attention
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 64, 5, 53, 53]          65,856
       BatchNorm3d-2        [-1, 64, 5, 53, 53]             128
       SamePadding-3        [-1, 64, 7, 55, 55]               0
         MaxPool3d-4        [-1, 64, 3, 27, 27]               0
       SamePadding-5        [-1, 64, 3, 27, 27]               0
            Conv3d-6       [-1, 256, 3, 27, 27]          16,384
       BatchNorm3d-7       [-1, 256, 3, 27, 27]             512
       BatchNorm3d-8       [-1, 256, 3, 27, 27]             512
       SamePadding-9        [-1, 64, 5, 27, 27]               0
           Conv3d-10        [-1, 64, 3, 27, 27]          12,288
      BatchNorm3d-11        [-1, 64, 3, 27, 27]             128
             ReLU-12        [-1, 64, 3, 27, 27]               0
      SamePadding-13        [-1, 64, 3, 29, 29]               0
          

In [12]:
def update_lr(optimizer, lr):
    for g in optimizer.param_groups:
        g['lr'] = lr
def update_mom(optimizer, mom):
    for g in optimizer.param_groups:
        g['momentum'] = mom

In [13]:
from Modular_code.lr_finder import *
import adabound
from RAdam.radam import RAdam

In [14]:
def lr_finder(model,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = RAdam(model.parameters(), lr=1e-6)
    lr_finder = LRFinder(model, optimizer_ft, criterion, device=device)
    lr_finder.range_test(train_loader, end_lr=1, num_iter=1000)
    lr_finder.reset()
    lr_finder.plot()

In [15]:
def mil_loss(anom_pred, normal_pred, k1 = 8e-5, k2 = 8e-5):
    batch_size = len(anom_pred)
    loss = []
    hl1 = []
    for j in range(batch_size):
        m1 = torch.max(anom_pred[j])
        m2 = torch.max(normal_pred[j])
        hl = 0
        for i in range(len(normal_pred[j])):
            m2 = max(0,1-m1+normal_pred[j][i])
            m3 = max(0,normal_pred[j][i]-torch.log2(m1))
            hl = hl + m2+m3
        hl = hl/len(normal_pred)
        h = max(0,1-m1+m2)
        hl1.append(h)
        smooth_loss = 0
        for i in range(len(anom_pred[j])-1):
            smooth_loss = smooth_loss + torch.pow(anom_pred[j][i]-anom_pred[j][i+1], 2) 
        smooth_loss = smooth_loss*k1
        sparse_loss = k2 * torch.sum(anom_pred[j])
        loss1 = hl + smooth_loss + sparse_loss
        loss.append(loss1)
    return torch.sum(torch.stack(loss).type(torch.FloatTensor)),torch.mean(torch.stack(hl1).type(torch.FloatTensor))

In [ ]:
num_epochs = 10
import face_recognition
batch_size = 10
optimizer = RAdam(model.parameters(), lr=0.01)
dl0 = DataLoader(train_data_0,batch_size = batch_size,shuffle=  True,num_workers = 4)
dl1 = DataLoader(train_data_1,batch_size = batch_size,shuffle=  True,num_workers = 4)
for epoch in range(num_epochs):
    print('')
    print("--- Epoch {} ---".format(epoch))
    phase1 = ['train']
    for phase in phase1:
        print('')
        print("--- Phase {} ---".format(phase))
        epoch_metrics = {"loss": [], "hl":[]}
        batch_i = 0
        if(phase == 'train'):
            model = model.train()
        else:
            model = model.eval()
        for j in range(300):
            optimizer.zero_grad()
            batch = []
            images_0 = next(iter(dl0))
            images_0 = torch.reshape(images_0,(-1,3,16,112,112))
            predictions_0 = model(images_0.type(torch.cuda.FloatTensor))
            del images_0
            images_1 = next(iter(dl1))
            images_1 = torch.reshape(images_1,(-1,3,16,112,112))
            predictions_1 = model(images_1.type(torch.cuda.FloatTensor))
            del images_1
            loss,h = mil_loss(predictions_0.squeeze().reshape(batch_size,9),predictions_1.squeeze().reshape(batch_size,9))
            loss.cuda().backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            try:
                epoch_metrics["hl"].append(np.mean(h.detach().cpu().numpy()))
            except:
                epoch_metrics["hl"].append(0.)
            sys.stdout.write(
                  "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f)] [Hinge Loss: (%f)]"
                  % (
                      epoch,
                      num_epochs,
                      j,
                      300,
                      loss.item(),
                      np.mean(epoch_metrics["loss"]),
                      np.mean(epoch_metrics["hl"])
                  )
              )
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            batch_i += 1
            if(j%100==0):
                torch.save(model.module.state_dict(),'./nn_crime.pt')
        print('')
        print('{} , Loss: {}'.format(phase,np.mean(epoch_metrics["loss"])))
        


--- Epoch 0 ---

--- Phase train ---
